In [1]:
import open3d as o3d
import numpy as np
# Load your point cloud
pcd = o3d.io.read_point_cloud("D:\\PROJECT\\Drone_reconstruction\\dense\\0\\fused.ply")

# Estimate normals if not already present
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1.0, max_nn=30))

# Poisson surface reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=14)

# (Optional) Remove low-density vertices (artifacts)
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)

o3d.visualization.draw_geometries([mesh])

#% Save
o3d.io.write_triangle_mesh("filled_surface.ply", mesh)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


KeyboardInterrupt: 

In [2]:
import open3d as o3d
import numpy as np

ply_path = r"filled_surface.ply"
mesh = o3d.io.read_triangle_mesh(ply_path)
o3d.visualization.draw_geometries([mesh])


In [ ]:
import open3d as o3d
ply_path = r"filled_surface.ply"
mesh = o3d.io.read_triangle_mesh(ply_path)
mesh.compute_vertex_normals()

renderer = o3d.visualization.rendering.OffscreenRenderer(1920, 1080)
scene = renderer.scene

scene.set_background([1, 1, 1, 1])  # vit bakgrund
scene.add_geometry("mesh", mesh, o3d.visualization.rendering.MaterialRecord())

# Lägg till ljus
scene.scene.set_lighting(o3d.visualization.rendering.Scene.LightingProfile.NO_SHADOWS, (0, 0, 0))

# Placera kameran
center = mesh.get_center()
bbox = mesh.get_axis_aligned_bounding_box()
cam = scene.camera
cam.look_at(center, center + [0, 0, 1], [0, 1, 0])

# Rendera bild
img = renderer.render_to_image()
o3d.io.write_image("screenshot.png", img)
